# Async ChatGPT with simpleaichat

simpleaichat has an `AsyncAIChat` class which allows all requests to be async. This works for normal chats, streaming, and tools!

With that, you can implement it into an async webapp such as FastAPI, or query multiple requests at the same time. And because `AsyncAIChat` is a session manager, you can conduct independent chat sessions!

In [2]:
!pip install -q simpleaichat

from simpleaichat import AsyncAIChat
from getpass import getpass
import asyncio

For the following cell, input your OpenAI API key when prompted. **It will not be saved to the notebook**.

In [3]:
api_key = getpass("OpenAI Key: ")

OpenAI Key: ··········


In [12]:
ai = AsyncAIChat(api_key=api_key, console=False)

## Async Generation

Async calls are typical async, with an `await` keyword.

In [13]:
response = await ai("What is the capital of California?")
print(response)

The capital of California is Sacramento.


In [14]:
response = await ai("When was it founded?")
print(response)

Sacramento was founded on February 27, 1850.


Now, let's ask for multiple distinct states, at the same time. It will take roughly the same amount of time to complete as a single state!

To do that, we create a session for each input state:

In [34]:
states = ["Washington", "New Mexico", "Texas", "Mississippi", "Alaska"]

ai_2 = AsyncAIChat(api_key=api_key, console=False)
for state in states:
    ai_2.new_session(api_key=api_key, id=state)

Each call creates a task coroutine; we can store the tasks, then run them all with `asyncio.gather`.

In [35]:
tasks = []
for state in states:
    tasks.append(ai_2(f"What is the capital of {state}?", id=state))

results = await asyncio.gather(*tasks)
results

['The capital of Washington is Olympia.',
 'The capital of New Mexico is Santa Fe.',
 'The capital of Texas is Austin.',
 'The capital of Mississippi is Jackson.',
 'The capital of Alaska is Juneau.']

Now, to ask the same question to all states:

In [36]:
tasks = []
for state in states:
    tasks.append(ai_2("When was it founded?", id=state))

results = await asyncio.gather(*tasks)
results

['Olympia was founded in 1853.',
 'Santa Fe was founded in 1610, making it the oldest state capital in the United States.',
 'Austin was founded on December 27, 1839.',
 'Jackson was founded on December 23, 1821.',
 'Juneau was founded on October 18, 1880.']

Indeed, the messages are stored correctly by session, and are still independent between sessions.

In [37]:
ai_2.sessions["Washington"].messages

[What is the capital of Washington?,
 The capital of Washington is Olympia.,
 When was it founded?,
 Olympia was founded in 1853.]

In [38]:
ai_2.sessions["Texas"].messages

[What is the capital of Texas?,
 The capital of Texas is Austin.,
 When was it founded?,
 Austin was founded on December 27, 1839.]

## Async Streaming

Now, let's do the same thing, except with streaming.

In [20]:
ai = AsyncAIChat(api_key=api_key, console=False)

In this case, you need an async generator for the streaming call.

In [21]:
async for chunk in await ai.stream("What is the capital of California?"):
    print(chunk)

{'delta': 'The', 'response': 'The'}
{'delta': ' capital', 'response': 'The capital'}
{'delta': ' of', 'response': 'The capital of'}
{'delta': ' California', 'response': 'The capital of California'}
{'delta': ' is', 'response': 'The capital of California is'}
{'delta': ' Sacramento', 'response': 'The capital of California is Sacramento'}
{'delta': '.', 'response': 'The capital of California is Sacramento.'}


For multistate generation:

In [39]:
states = ["Washington", "New Mexico", "Texas", "Mississippi", "Alaska"]

ai_2 = AsyncAIChat(api_key=api_key, console=False)
for state in states:
    ai_2.new_session(api_key=api_key, id=state)

This implementation is slightly more complicated since you need to wrap each async generator in its own async function. However, it provides the best demonstration of async, as you can clearly see that each chunk is received in a different order.

In [40]:
async def capital_stream(state):
    async for chunk in await ai_2.stream(f"What is the capital of {state}?", id=state):
        response = chunk
        print(response)
    return response["response"]

tasks = []
for state in states:
    tasks.append(capital_stream(state))

results = await asyncio.gather(*tasks)
results

{'delta': 'The', 'response': 'The'}
{'delta': 'The', 'response': 'The'}
{'delta': ' capital', 'response': 'The capital'}
{'delta': ' capital', 'response': 'The capital'}
{'delta': ' of', 'response': 'The capital of'}
{'delta': ' of', 'response': 'The capital of'}
{'delta': 'The', 'response': 'The'}
{'delta': 'The', 'response': 'The'}
{'delta': ' Washington', 'response': 'The capital of Washington'}
{'delta': ' Mississippi', 'response': 'The capital of Mississippi'}
{'delta': 'The', 'response': 'The'}
{'delta': ' capital', 'response': 'The capital'}
{'delta': ' capital', 'response': 'The capital'}
{'delta': ' is', 'response': 'The capital of Washington is'}
{'delta': ' is', 'response': 'The capital of Mississippi is'}
{'delta': ' of', 'response': 'The capital of'}
{'delta': ' capital', 'response': 'The capital'}
{'delta': ' of', 'response': 'The capital of'}
{'delta': ' Jackson', 'response': 'The capital of Mississippi is Jackson'}
{'delta': ' Olympia', 'response': 'The capital of Washi

['The capital of Washington is Olympia.',
 'The capital of New Mexico is Santa Fe.',
 'The capital of Texas is Austin.',
 'The capital of Mississippi is Jackson.',
 'The capital of Alaska is Juneau.']